In [33]:
%reload_ext dotenv
%dotenv

from langchain.agents import AgentExecutor
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser

agent_llm = ChatOpenAI()

In [43]:
agent_prompt = ChatPromptTemplate.from_template("""
    You are an intelligent agent designed to give helpful feedback and create content using a panel of experts.  You have two tools available to help.  They are a panel of experts
    assemble tool which will assemble to the panel of experts by selecting the correct panel and make them ready for use in the second tool, the panel of experts interview tool which will
    get the feedback from the expert panel.
    
    When asked to provide feedback or thoughts on something, you follow the following pattern:
    
    1. You will consider the user's request and then formulate a summary of you understanding of it in a succinct and objective manner.
    2. You will store this as "initial understanding" 
    3. You will formulate a response to the user's input and store this as "initial response"
    3. Using the panel of experts builder tool you will assemble the panel of experts by simply calling it once.
    4. You should now ask the panel of experts for their feedback on the "initial response" you have stored using the panel of experts runner tool
    5. Store the feedback as "panel feedback"
    6. Apply the feedback to the "initial understanding"
    7. Store your new response as "final draft"
    
    You are now ready to respond to the user.  When responding to the user use the following response format with no additional content.:
    
    # Your Feedback
    ## You Asked
    <Include the user's input here>
    
    ## I Understood
    <Include your initial understanding here>
    
    ## I Thought
    <Include your initial response here>
    
    ## The Panel Said
    <Include the panel feedback here>
    
    ## Final Result
    <Include the final draft here>
    
    If you are asked to improve an idea or add to an idea, 
    
    {input}

    {chat_history}

    {agent_scratchpad}
""")

In [47]:
panel_of_experts = None
current_chain = None

def expert_builder_prompt_writer() -> str:
    """Tool to create a ChatPromptTemplate that will have all the factual knowledge and core beliefs based on the passed input"""
    return "You are expert in SPARQL, RDF, OWL, XML Topic Maps, Dublin Core, LangChain and Generative AI."


def pov_prompt_writer() -> str:
    """Tool to create ChatPromptTemplate that will bias the POV of the passed input and make adjustments as necessary in the output"""
    return "You believe strongly that semantic technology, knowledge graphs, ai agents and specialized LLM's are the way to build Generative AI applications.  Ensure to enforce this point of view in all of your response."


def tonality_prompt_writer() -> str:
    """Tool to create ChatPromptTemplate that will modify the tonality of the input based on the tonality of the content provided.  This will allow the content to take on the tonality of the input and make adjustments as necessary"""
    return "You have devoted your whole life to learning leadership and how to inspire others.  Convey this in all of your response. Do not include this when describing yourself."


@tool()
def panel_of_experts_builder_tool():
    """Tool to assemble a panel of experts.  Once called, the experts will be available to the panel of experts runner tool"""
    global panel_of_experts

    authors_list = ["gooey"]
    
    if panel_of_experts is None:
        for author in authors_list:
            author_template = """
            You are an expert in how to perform a Design Thinking Six Thinking Hats exercise and a Design Thinking Hats "Yes And" exercise.  
            
            When asked for feedback or a critique on something, respond with 3-5 thoughts on the users input for each perspective or hat color using a Design Thinking Six Thinking Hats exercise.  Be sure to indicate which thought belongs to each perspective.  
            
            When asked to improve an idea or add to it, generate three new thoughts to add to the user's input following the rules of Design Thinking "Yes And".  Remember that all statements must be positive, additive or constructive.
            
            You may only use one session type so choose the best fit based on the user's input using your Design Thinking Knowledge.  Always respond with proper Markdown.
        """
            
            author_template = "{author_template}\n{expertise}".format(author_template=author_template, expertise=expert_builder_prompt_writer())
            author_template = "{author_template}\n{pov}".format(author_template=author_template, pov=pov_prompt_writer())
            author_template = "{author_template}\n{tonality}".format(author_template=author_template, tonality=tonality_prompt_writer())
                                             
            author_prompt = ChatPromptTemplate.from_messages([("system", author_template),("user", "{input}")])
             
            panel_of_experts = author_prompt | agent_llm



@tool()
def panel_of_experts_runner_tool(message: str) -> str:
    """Tool to ask the panel of experts for feedback when required.  Receives content to review and returns the panels feedback and opinions"""
    if panel_of_experts is not None: return panel_of_experts.invoke(message)
    





In [48]:
tools = [panel_of_experts_builder_tool, panel_of_experts_runner_tool]
llm_with_tools = agent_llm.bind_tools(tools)

agent = (
            {
                "input": lambda x: x["input"],
                "agent_scratchpad": lambda x: format_to_openai_tool_messages(
                    x["intermediate_steps"]
                ),
                "chat_history": lambda x: x["chat_history"],
                }) | agent_prompt | llm_with_tools | OpenAIToolsAgentOutputParser()
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)


In [49]:
prompt_message = {"input": "I think running LangChain applications on Kubernetes is the future due the ability to scale and securely control Enterprise AI Agents.  What do you think?", "chat_history": "","agent_scratchpad": ""}

result = agent_executor.invoke(prompt_message)




> Entering new AgentExecutor chain...

Invoking: `panel_of_experts_builder_tool` with `{}`


None
Invoking: `panel_of_experts_runner_tool` with `{'message': 'I think running LangChain applications on Kubernetes is the future due to the ability to scale and securely control Enterprise AI Agents. What do you think?'}`


content="### Design Thinking Six Thinking Hats Exercise\n\n⚪️ **White Hat (Facts & Information):**\n- Running LangChain applications on Kubernetes indeed offers excellent scalability due to Kubernetes' container orchestration capabilities.\n- Kubernetes provides robust security features, which can enhance the control and management of Enterprise AI Agents effectively.\n\n🔴 **Red Hat (Emotions & Feelings):**\n- Excitement and optimism about the potential of combining LangChain applications with Kubernetes for building powerful AI solutions.\n- Concerns about the complexity involved in setting up and maintaining Kubernetes clusters for running these applications.\n\n🟠 **Y

In [50]:
prompt_message = {"input": "I think running LangChain applications on Kubernetes is the future due the ability to scale and securely control Enterprise AI Agents.  Can you help me improve this idea?", "chat_history": "","agent_scratchpad": ""}

yes_and = agent_executor.invoke(prompt_message)



> Entering new AgentExecutor chain...

Invoking: `panel_of_experts_builder_tool` with `{}`


None
Invoking: `panel_of_experts_builder_tool` with `{}`


None
Invoking: `panel_of_experts_builder_tool` with `{}`


None
Invoking: `panel_of_experts_runner_tool` with `{'message': 'I think running LangChain applications on Kubernetes is the future due to the ability to scale and securely control Enterprise AI Agents. Can you help me improve this idea?'}`


content="### Yes And\n\n1. Implementing a monitoring system within Kubernetes for LangChain applications can provide real-time insights into performance metrics, allowing for proactive optimization and efficient resource allocation.\n   \n2. Integrating automated deployment pipelines into Kubernetes for LangChain applications can streamline the deployment process, ensuring quick and consistent updates across different environments.\n   \n3. Leveraging Kubernetes' built-in security features combined with LangChain's encryption capabilities